# News Extraction From Csv

In [6]:
# Modul Dasar
import numpy as np               # Operasi matematika dan array
import pandas as pd              # Struktur data tabular (DataFrame)
import matplotlib.pyplot as plt  # Visualisasi data
import seaborn as sns            # Visualisasi data tambahan
import re

# Statistik dan Machine Learning
from scipy import stats          # Statistik
from sklearn import datasets     # Datasets untuk machine learning
from sklearn.model_selection import train_test_split  # Pembagian data train-test
from sklearn.preprocessing import StandardScaler     # Normalisasi data
from sklearn.linear_model import LinearRegression    # Regresi linear
from sklearn.tree import DecisionTreeClassifier      # Klasifikasi menggunakan Decision Tree
from sklearn.cluster import KMeans                    # KMeans clustering

# Evaluasi Model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Evaluasi performa model

# Visualisasi Machine Learning
from sklearn.tree import plot_tree    # Visualisasi struktur Decision Tree
from sklearn.metrics import roc_curve, auc  # Kurva ROC dan AUC

In [5]:
data = pd.read_csv(r"C:\Users\ZADO\Documents\Kampus\TA\Forecast by Sentiment\stock_forecast_sentiment\result_fix.csv",sep=";")
data.head()


,tanggal,judul,link,isi
0,"Kompas.com - 21/05/2013, 08:17 WIB",IHSG Bakal Hadapi Tekanan Ambil Untung,https://money.kompas.com/read/2013/05/21/08175...,IHSG Bakal Hadapi Tekanan Ambil Untung IHSG B...
1,"Kompas.com - 21/05/2013, 08:27 WIB",Proyeksi IHSG di Rentang 5.185-5.235,https://money.kompas.com/read/2013/05/21/08270...,Proyeksi IHSG di Rentang 5.185-5.235 Proyeksi ...
2,"Kompas.com - 21/05/2013, 09:21 WIB",IHSG Dibuka Naik di Tengah Pelemahan Asia,https://money.kompas.com/read/2013/05/21/09212...,IHSG Dibuka Naik di Tengah Pelemahan Asia IHSG...
3,"Kompas.com - 21/05/2013, 13:04 WIB","Bursa Regional Merah, IHSG Turun 29 Poin",https://money.kompas.com/read/2013/05/21/13042...,"Bursa Regional Merah, IHSG Turun 29 Poin Bursa..."
4,"Kompas.com - 21/05/2013, 16:35 WIB","Asing Lepas Saham, IHSG Melorot 26 Poin",https://money.kompas.com/read/2013/05/21/16354...,"Asing Lepas Saham, IHSG Melorot 26 Poin Asing ..."


In [7]:
# Fungsi untuk mengekstrak tanggal dan waktu dari URL
def extract_date_time(url):
    match = re.search(r'(\d{4}/\d{2}/\d{2})/(\d{8})/', url)
    if match:
        date_str = match.group(1)  # Tanggal
        time_str = match.group(2)   # Waktu dalam WIB
        # Mengubah waktu WIB ke format waktu
        hour = int(time_str[:2])
        minute = int(time_str[2:4])
        second = int(time_str[4:6])
        # Mengubah waktu WIB ke format tanggal Python
        return pd.to_datetime(f"{date_str} {hour:02}:{minute:02}:{second:02}")
    return None

# Buat kolom tanggal baru
data['tanggal_fix'] = data['link'].apply(extract_date_time)

In [ ]:
data = data.drop(["tanggal"],axis=1)


In [12]:
data.head()

,judul,link,isi,tanggal_fix
0,IHSG Bakal Hadapi Tekanan Ambil Untung,https://money.kompas.com/read/2013/05/21/08175...,IHSG Bakal Hadapi Tekanan Ambil Untung IHSG B...,2013-05-21 08:17:59
1,Proyeksi IHSG di Rentang 5.185-5.235,https://money.kompas.com/read/2013/05/21/08270...,Proyeksi IHSG di Rentang 5.185-5.235 Proyeksi ...,2013-05-21 08:27:02
2,IHSG Dibuka Naik di Tengah Pelemahan Asia,https://money.kompas.com/read/2013/05/21/09212...,IHSG Dibuka Naik di Tengah Pelemahan Asia IHSG...,2013-05-21 09:21:29
3,"Bursa Regional Merah, IHSG Turun 29 Poin",https://money.kompas.com/read/2013/05/21/13042...,"Bursa Regional Merah, IHSG Turun 29 Poin Bursa...",2013-05-21 13:04:20
4,"Asing Lepas Saham, IHSG Melorot 26 Poin",https://money.kompas.com/read/2013/05/21/16354...,"Asing Lepas Saham, IHSG Melorot 26 Poin Asing ...",2013-05-21 16:35:47


In [ ]:
def clean_text(text):
    # Hapus kalimat pertama
    text = re.sub(r'^[^.]*\.\s*', '', text)
    # Hapus copyright di paling bawah
    text = re.sub(r'Copyright.*$', '', text, flags=re.MULTILINE)
    return text.strip()

# Terapkan fungsi pada kolom isi
data['cleaned_isi'] = data['isi'].apply(clean_text)